In [20]:
from dotenv import load_dotenv
from supabase import create_client, Client
import os

load_dotenv()

supabase_url = os.getenv("SUPABASE_URL")
supabase_key = os.getenv("SUPABASE_KEY")

supabase: Client = create_client(supabase_url, supabase_key)

In [21]:
import pandas as pd

In [22]:
response = (
    supabase.table("Payments")
    .select("*")
    .csv()
    .execute()
)

In [23]:
response.data

'id,created_at,uuid,product_name,quantity,price,purchase_date,tags\n2,"2025-04-11 17:16:10.107133+00",9e683645-8dfe-4954-b041-0ad4a1723174,test,10,100,2009-02-05,"{test,MegaTest}"\n3,"2025-04-11 17:17:29.596742+00",9e683645-8dfe-4954-b041-0ad4a1723174,яблоко,10,30,2025-04-11,"{test,food}"\n4,"2025-04-11 17:17:58.008047+00",9e683645-8dfe-4954-b041-0ad4a1723174,дом,5,100000000,2025-04-10,"{test,home}"\n5,"2025-04-11 17:18:17.36416+00",9e683645-8dfe-4954-b041-0ad4a1723174,banana,100,200,2025-04-02,"{test,food}"\n6,"2025-04-11 17:18:33.797878+00",9e683645-8dfe-4954-b041-0ad4a1723174,banana,500,1000,2025-04-11,"{food,test}"\n7,"2025-04-11 17:19:38.887289+00",9e683645-8dfe-4954-b041-0ad4a1723174,banana,180,500,2025-04-12,"{test,food}"\n8,"2025-04-11 20:24:01.007393+00",9e683645-8dfe-4954-b041-0ad4a1723174,Чипсы,10,150,2025-04-11,"{food,fastfood}"'

In [24]:
from io import StringIO

In [25]:
df = pd.read_csv(StringIO(response.data), index_col=0)
df.drop(columns=['created_at'], inplace=True)
df

,uuid,product_name,quantity,price,purchase_date,tags
id,,,,,,
2,9e683645-8dfe-4954-b041-0ad4a1723174,test,10,100,2009-02-05,"{test,MegaTest}"
3,9e683645-8dfe-4954-b041-0ad4a1723174,яблоко,10,30,2025-04-11,"{test,food}"
4,9e683645-8dfe-4954-b041-0ad4a1723174,дом,5,100000000,2025-04-10,"{test,home}"
5,9e683645-8dfe-4954-b041-0ad4a1723174,banana,100,200,2025-04-02,"{test,food}"
6,9e683645-8dfe-4954-b041-0ad4a1723174,banana,500,1000,2025-04-11,"{food,test}"
7,9e683645-8dfe-4954-b041-0ad4a1723174,banana,180,500,2025-04-12,"{test,food}"
8,9e683645-8dfe-4954-b041-0ad4a1723174,Чипсы,10,150,2025-04-11,"{food,fastfood}"


In [26]:
test_uuid = "9e683645-8dfe-4954-b041-0ad4a1723174"

In [33]:
tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
tmp = tmp.groupby(['product_name', 'purchase_date']).sum()

tmp

quantity      price
product_name purchase_date                     
banana       2025-04-02          100        200
             2025-04-11          500       1000
             2025-04-12          180        500
test         2009-02-05           10        100
Чипсы        2025-04-11           10        150
дом          2025-04-10            5  100000000
яблоко       2025-04-11           10         30

In [37]:
import pandas as pd
import json

# Предположим, что tmp уже существует
# tmp = df[df['uuid'] == test_uuid][['product_name', 'quantity', 'price', 'purchase_date']]
# tmp = tmp.groupby(['product_name', 'purchase_date']).sum()

# Сброс индекса, чтобы сделать product_name и purchase_date обычными колонками
tmp_reset = tmp.reset_index()

# Создание списка уникальных дат
labels = tmp_reset['purchase_date'].unique().tolist()

# Подготовка данных для Quantity
quantity_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    quantity_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['quantity'].reindex(labels, fill_value=0).tolist()
    })

# Подготовка данных для Price
price_datasets = []
for product_name, group in tmp_reset.groupby('product_name'):
    price_datasets.append({
        "label": product_name,
        "data": group.set_index('purchase_date')['price'].reindex(labels, fill_value=0).tolist()
    })

# Формирование финального JSON
chart_data = {
    "quantity": {
        "labels": labels,
        "datasets": quantity_datasets
    },
    "price": {
        "labels": labels,
        "datasets": price_datasets
    }
}

# Вывод JSON в читаемом формате
print(json.dumps(chart_data, indent=4))

{
    "quantity": {
        "labels": [
            "2025-04-02",
            "2025-04-11",
            "2025-04-12",
            "2009-02-05",
            "2025-04-10"
        ],
        "datasets": [
            {
                "label": "banana",
                "data": [
                    100,
                    500,
                    180,
                    0,
                    0
                ]
            },
            {
                "label": "test",
                "data": [
                    0,
                    0,
                    0,
                    10,
                    0
                ]
            },
            {
                "label": "\u0427\u0438\u043f\u0441\u044b",
                "data": [
                    0,
                    10,
                    0,
                    0,
                    0
                ]
            },
            {
                "label": "\u0434\u043e\u043c",
                "data": [
               